# Avatarify Colab Server

Este bloco de notas Colab é para executar o servidor de renderização Avatarify. Ele permite que você execute o Avatarify em seu computador **sem GPU** desta forma:

1. Quando este bloco de notas é executado, ele começa a escutar as solicitações de entrada do seu computador;
1. Você inicia o cliente em seu computador e ele se conecta ao notebook e começa a enviar solicitações;
1. Este notebook recebe as solicitações do seu computador, renderiza imagens de avatar e as envia de volta;

Para isso, todo o trabalho pesado é transferido do seu computador para este notebook para que você não precise mais ter um hardware beafy no seu PC.

## Inicie o servidor
Execute as células abaixo (Shift + Enter) sequencialmente e preste atenção às dicas e instruções incluídas neste bloco de notas.

No final, você receberá um comando para executar o cliente em seu computador.

## Inicie o cliente

Certifique-se de que instalou a versão mais recente do Avatarify no seu computador. Consulte o [LEIA-ME](https://github.com/alievk/avatarify#install) para obter as instruções.

Quando estiver pronto execute este notebook e obtenha o comando para rodar o cliente em seu computador.

### Detalhes técnicos

O cliente em seu computador se conecta ao servidor via túnel TCP `ngrok` ou túnel` ssh` reverso.

O `ngrok`, embora fácil de usar, pode induzir um atraso considerável na rede, variando de dezenas de milissegundos a um segundo. Isso pode levar a uma experiência ruim.

Uma conexão mais estável poderia ser estabelecida usando um túnel reverso `ssh` para um host com um IP público, como uma instância AWS` t3.micro` (gratuita). Este bloco de notas fornece um script para a criação de um túnel, mas o lançamento de uma instância em uma nuvem é por sua conta (encontre o manual abaixo).

# Instalar

### Avatarify
#### Siga os passos abaixo para baixar o avatarify e os itens de grafico

In [ ]:
!cd /content
!rm -rf *

In [ ]:
!git clone https://github.com/DybaTubeYT1/Avatarify-Python-BR

Cloning into 'Avatarify-Python-BR'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
cd avatarify

/content/avatarify


In [ ]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy pyyaml==5.1

Cloning into 'fomm'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 58.16 MiB | 40.65 MiB/s, done.
Resolving deltas: 100% (108/108), done.
     |████████████████████████████████| 276kB 19.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=71f4a4b4f5638a9a27e334453a307ee8031c26ac2dad4e44d3045ac3aa1987c0
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!scripts/download_data.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  228M  100  228M    0     0  53.2M      0  0:00:04  0:00:04 --:--:-- 53.2M
Expected checksum: 8a45a24037871c045fbb8a6a8aa95ebc
Found checksum:    8a45a24037871c045fbb8a6a8aa95ebc  vox-adv-cpk.pth.tar


### ngrok
Siga abaixo para baixar o ngok. Você precisa registrar no ngok para conseguir o tunel, esse authtoken é **IMPORTANTE**


In [ ]:
# Download ngrok
!scripts/get_ngrok.sh

ngrok is not found, installing...
Done!


# Execute
Comece aqui se o ambiente foi reiniciado depois da instalação ou caso você tenha baixado tudo daqui

In [ ]:
cd /content/avatarify

/content/avatarify


In [ ]:
#!git pull origin

In [ ]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [ ]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Comece o trabalhador (inteligencia artificial)


In [ ]:
# (Re)Start the worker
with open('/tmp/run.txt', 'w') as f:
  ps = Popen(
      shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port} --no-vcam --no-conda'),
      stdout=f, stderr=f)
  time.sleep(3)

This command should print lines if the worker is successfully started

In [ ]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep | tee /tmp/ps_run
!if [[ $(cat /tmp/ps_run | wc -l) == "0" ]]; then echo "Worker failed to start"; cat /tmp/run.txt; else echo "Worker started"; fi

root         133 18.2  1.4 1973252 196720 ?      D    10:17   0:00 python3 afy/cam_fomm.py --config fomm/config/vox-adv-256.yaml --checkpoint vox-adv-cpk.pth.tar --virt-cam 9 --relative --adapt_scale --is-worker --in-port 5557 --out-port 5558 --no-stream
Worker started


# Abre o tunel do ngok

#### Ganhe o authtoken do Ngok (falei que era importante)
Vá para https://dashboard.ngrok.com/auth/your-authtoken (registre CASO CE VOCÊ AINDA NAO REGISTROU), copie o token e paste aq

In [ ]:
# Paste your authtoken here in quotes
authtoken = "1pcD6A9MspWbll9x0R4kkMiu5Wu_384BcYW7pe8b8TdLPDqTL"

**Set** Escolha sua ***Região***

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [ ]:
# Escolha sua região aqui
region = "sa"

In [ ]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [ ]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [ ]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
  print("Tunnel opened")
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

tcp://0.tcp.sa.ngrok.io:18440 -> 5558 [output]
tcp://0.tcp.sa.ngrok.io:10152 -> 5557 [input]
Tunnel opened


### [Optional] AWS proxy (esse vou deixar em inglês pois nao vai ser necessário no windows)
Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command (substitute `run_mac.sh` with `run_windows.bat` or `run.sh`)
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [ ]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Inicie o cliente
Quando você executa a célula abaixo, ela imprime um comando. Execute este comando no seu computador:

1. Abra um terminal (no Windows, abra o `Anaconda Prompt`);
2. Mude o diretório de trabalho para o diretório `avatarify`: </br>
* Windows (altere `C: \ path \ to \ avatarify` para o seu caminho) </br>
`cd C: \ caminho \ para \ avatarify` </br> </br>
* Mac / Linux (altere `/ path / to / avatarify` para o seu caminho) </br>
`cd / caminho / para / avatarify`
3. Copie e cole no terminal o comando abaixo e execute;
4. A conexão pode demorar algum tempo (geralmente até 10 segundos). Se a janela de visualização não aparecer em um ou dois minutos, procure os erros acima neste bloco de notas e relate nos [problemas] (https://github.com/alievk/avatarify/issues) ou [Slack] (https : //join.slack.com/t/avatarify/shared_invite/zt-dyoqy8tc-~4U2ObQ6WoxuwSaWKKVOgg).

In [ ]:
print('Copie e cole no terminal o comando abaixo e execute (pressione Enter)\n')
print('Mac:')
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nWindows:')
print(f'run_windows.bat --is-client --in-addr {in_addr} --out-addr {out_addr}')
print('\nLinux:')
print(f'./run.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

Copy-paste to the terminal the command below and run (press Enter)

Mac:
./run_mac.sh --is-client --in-addr tcp://0.tcp.sa.ngrok.io:10152 --out-addr tcp://0.tcp.sa.ngrok.io:18440

Windows:
run_windows.bat --is-client --in-addr tcp://0.tcp.sa.ngrok.io:10152 --out-addr tcp://0.tcp.sa.ngrok.io:18440

Linux:
./run.sh --is-client --in-addr tcp://0.tcp.sa.ngrok.io:10152 --out-addr tcp://0.tcp.sa.ngrok.io:18440


# Logs

Se alguma coisa não funcionou veja o log aqui (***Nesses codigos***)

In [ ]:
#@title
!cat ./var/log/cam_fomm.log | head -100

In [ ]:
#@title
!cat ./var/log/recv_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/predictor_worker.log | tail -100

In [ ]:
#@title
!cat ./var/log/send_worker.log | tail -100